In [1]:
import tensorflow as tf
from tensorflow.contrib.model_pruning.python import pruning
from tensorflow.contrib.model_pruning.python.layers import layers
import keras
from utils import *

Using TensorFlow backend.


In [2]:
epochs = 50
epochs_prune = 5
batch_size = 250 # Entire training set
model_path_unpruned = "Model_Saves/Unpruned{}.ckpt"
model_path_pruned = "Model_Saves/Pruned{}.ckpt"
NUM_CLASS = 10

In [3]:
#Load Dataset
X_train0, y_train = prepare_dataset(data_dir, 'train')
X_test0, y_test = prepare_dataset(data_dir, 'test')
t = int(time.time())

#Normalizing
mean = np.mean(X_train0,axis=(0,1,2,3))
std = np.std(X_train0,axis=(0,1,2,3))
np.save('mean',mean)
np.save('std',std)
X_train = z_normalization(X_train0, mean, std)
X_test = z_normalization(X_test0, mean, std)

#Labels to binary
y_train_binary = keras.utils.to_categorical(y_train,num_classes)
y_test_binary = keras.utils.to_categorical(y_test,num_classes)

batches = int(len(X_train) / batch_size)
batches_test = int(len(X_test) / batch_size)
print(batches)

Loading numpy
Loading numpy
200


In [4]:
def sample_batch(dataset, labels, batch_size):
    N = dataset.shape[0]
    indices = np.random.randint(N, size=batch_size)
    x_epoch = dataset[indices]
    y_epoch = labels[indices]
    return x_epoch, y_epoch

def test_batch(dataset, labels, batch_size):
    for index, offset in enumerate(range(0, dataset.shape[0], batch_size)):
        x_epoch, y_epoch = np.array(dataset[offset: offset + batch_size,:]), np.array(labels[offset: offset +  batch_size])
    return x_epoch, y_epoch

In [5]:
tf.reset_default_graph()

image = tf.placeholder(name='images', dtype=tf.float32, shape=[None, 32, 32, 3])
label = tf.placeholder(name='fine_labels', dtype=tf.int32, shape=[None, 10])

# # Define the model
# layer1 = layers.masked_fully_connected(image, 300)
# layer2 = layers.masked_fully_connected(layer1, 300)
# logits = layers.masked_fully_connected(layer2, 10)

# Define the model
# layer1 = layers.masked_conv2d(image, 300, kernel_size=2)
# layer2 = layers.masked_conv2d(layer1, 300, kernel_size=2)
# logits = layers.masked_fully_connected(layer2, 10)

_=image
_ = layers.masked_conv2d(_, 96, (3, 3), 1, 'SAME')
_ = tf.layers.batch_normalization(_, name='norm1-1')
_ = layers.masked_conv2d(_, 96, (3, 3), 1, 'SAME')
_ = tf.layers.batch_normalization(_, name='norm1-2')
_ = tf.layers.max_pooling2d(_, (3, 3), 2, 'SAME',name='pool1')
_ = layers.masked_conv2d(_, 192, (3, 3), 1, 'SAME')
_ = tf.layers.batch_normalization(_, name='norm2-1')
_ = layers.masked_conv2d(_, 192, (3, 3), 1, 'SAME')
_ = tf.layers.batch_normalization(_, name='norm2-2')
_ = tf.layers.max_pooling2d(_, (3, 3), 2, 'SAME', name='pool2')
_ = layers.masked_conv2d(_, 192, (3, 3), 1, 'VALID')
_ = tf.layers.batch_normalization(_, name='norm3')
_ = layers.masked_conv2d(_, 192, (1, 1), 1)
_ = tf.layers.batch_normalization(_, name='norm4')
_ = layers.masked_conv2d(_, 10, (1, 1), 1)
_ = tf.layers.batch_normalization(_, name='norm5')
_ = tf.layers.average_pooling2d(_, (6,6), 1, name='avg_pool')
y = _
logits = tf.reshape(y,[tf.shape(y)[0],10])


# Create global step variable (needed for pruning)
global_step = tf.train.get_or_create_global_step()
reset_global_step_op = tf.assign(global_step, 0)

# Loss function

loss = tf.losses.softmax_cross_entropy(label, logits)

# Training op, the global step is critical here, make sure it matches the one used in pruning later
# running this operation increments the global_step
train_op = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(loss, global_step=global_step)
# train_op = tf.train.GradientDescentOptimizer(
#         learning_rate=0.001,
#         # beta1=0.9,
#         # beta2=0.999,
#         # epsilon=1e-08,
#         use_locking=False,
#         name='GD'
#     ).minimize(loss, global_step=global_step)

# Accuracy ops
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(label, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
accuracy_5 = tf.reduce_mean(tf.cast(tf.nn.in_top_k(predictions=logits, targets=tf.argmax(label, 1), k=5), tf.float32))


In [6]:
# Get, Print, and Edit Pruning Hyperparameters
pruning_hparams = pruning.get_pruning_hparams()
print("Pruning Hyperparameters:", pruning_hparams)

# Change hyperparameters to meet our needs
pruning_hparams.begin_pruning_step = 0
pruning_hparams.end_pruning_step = 250
pruning_hparams.pruning_frequency = 1
pruning_hparams.sparsity_function_end_step = 250
pruning_hparams.target_sparsity = .5

# Create a pruning object using the pruning specification, sparsity seems to have priority over the hparam
p = pruning.Pruning(pruning_hparams, global_step=global_step) #sparsity=.5)
prune_op = p.conditional_mask_update_op()

# Create a saver for writing training checkpoints.
saver = tf.train.Saver()



Pruning Hyperparameters: [('begin_pruning_step', 0), ('block_height', 1), ('block_pooling_function', 'AVG'), ('block_width', 1), ('do_not_prune', ['']), ('end_pruning_step', -1), ('initial_sparsity', 0), ('name', 'model_pruning'), ('nbins', 256), ('pruning_frequency', 10), ('sparsity_function_begin_step', 0), ('sparsity_function_end_step', 100), ('sparsity_function_exponent', 3), ('target_sparsity', 0.5), ('threshold_decay', 0.9), ('use_tpu', False)]
INFO:tensorflow:Updating masks.


In [8]:
with tf.Session() as sess:

    # Uncomment the following if you don't have a trained model yet
    sess.run(tf.global_variables_initializer())

    # Train the model before pruning (optional)
    for epoch in range(epochs):
        for batch in range(batches):
            batch_xs, batch_ys = sample_batch(X_train, y_train_binary, batch_size)
            sess.run(train_op, feed_dict={image: batch_xs, label: batch_ys})

        # Calculate Test Accuracy every 10 epochs
        if (epoch+1) % 5 == 0:
            acc_print = 0
            acc_print_5 = 0
            for index, offset in enumerate(range(0, X_test.shape[0], batch_size)):
                batch_xt = np.array(X_test[offset: offset + batch_size,:]) 
                batch_yt = np.array(y_test_binary[offset: offset +  batch_size])
#             for batch in range(batches_test):
#                 batch_xt, batch_yt = test_batch(X_test, y_test_binary, batch_size)
                acc_print += sess.run(accuracy, feed_dict={image: batch_xt, label: batch_yt})
                acc_print_5 += sess.run(accuracy_5, feed_dict={image: batch_xt, label: batch_yt})
            print("Un-pruned model step %d test accuracy %g" % (epoch, acc_print/batches_test))
            print("Un-pruned model step %d test top5-accuracy %g" % (epoch, acc_print_5/batches_test))
            
            # Saves the model before pruning
            saver.save(sess, model_path_unpruned.format(epoch))
        print(epoch)
            
        
    print("Top-1 Pre-Pruning accuracy:", acc_print/batches_test)
    print("Top-5 Pre-Pruning accuracy:", acc_print_5/batches_test)
    print("Sparsity of layers (should be 0)", sess.run(tf.contrib.model_pruning.get_weight_sparsity()))

    

    # Resets the session and restores the saved model
    sess.run(tf.initialize_all_variables())
    saver.restore(sess, model_path_unpruned.format(int(acc_print/batches)))

    # Reset the global step counter and begin pruning
    sess.run(reset_global_step_op)
    for epoch in range(epochs_prune):
        for batch in range(batches):
            batch_xs, batch_ys = sample_batch(X_train, y_train_binary, batch_size)
            # Prune and retrain
            sess.run(prune_op)
            sess.run(train_op, feed_dict={image: batch_xs, label: batch_ys})

        # Calculate Test Accuracy every 10 epochs
        if epoch % 1 == 0:
            acc_print = 0
            acc_print_5 = 0
            for batch in range(batches):
                batch_xt, batch_yt = sample_batch(X_test, y_test_binary, batch_size)
                acc_print += sess.run(accuracy, feed_dict={image: batch_xt, label: batch_yt})
            
            print("Pruned model step %d test accuracy %g" % (epoch, acc_print/batches))
            print("Weight sparsities:", sess.run(tf.contrib.model_pruning.get_weight_sparsity()))
        print(epoch) 
        
           # acc_print = sess.run(accuracy, feed_dict={image: mnist.test.images, label: mnist.test.labels})

    # Saves the model after pruning
    saver.save(sess, model_path_pruned.format(int(acc_print/batches)))

    # Print final accuracy
    #acc_print = sess.run(accuracy, feed_dict={image: mnist.test.images, label: mnist.test.labels})
    print("Final accuracy:", acc_print/batches)
    print("Final sparsity by layer (should be 0)", sess.run(tf.contrib.model_pruning.get_weight_sparsity()))

Un-pruned model step 0 test accuracy 0.3486
Un-pruned model step 0 test top5-accuracy 0.8616
0
1
2
3
4
Un-pruned model step 5 test accuracy 0.7723
Un-pruned model step 5 test top5-accuracy 0.9859
5
6
7
8
9
Un-pruned model step 10 test accuracy 0.8091
Un-pruned model step 10 test top5-accuracy 0.9853
10
11
12
13
14
Un-pruned model step 15 test accuracy 0.8205
Un-pruned model step 15 test top5-accuracy 0.9906
15
16
17
18
19
Un-pruned model step 20 test accuracy 0.8163
Un-pruned model step 20 test top5-accuracy 0.9885
20
21
22
23
24
Un-pruned model step 25 test accuracy 0.8299
Un-pruned model step 25 test top5-accuracy 0.9915
25
26
27
28
29
Un-pruned model step 30 test accuracy 0.826
Un-pruned model step 30 test top5-accuracy 0.9891
30
31
32
33
34
Un-pruned model step 35 test accuracy 0.8328
Un-pruned model step 35 test top5-accuracy 0.9894
35
36
37
38
39
Un-pruned model step 40 test accuracy 0.8321
Un-pruned model step 40 test top5-accuracy 0.9888
40
41
42
43
44
Un-pruned model step 45 t

NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for Model_Saves/Unpruned0.ckpt
	 [[Node: save/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, ..., DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/RestoreV2/tensor_names, save/RestoreV2/shape_and_slices)]]
	 [[Node: save/RestoreV2/_353 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device_incarnation=1, tensor_name="edge_84_save/RestoreV2", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'save/RestoreV2', defined at:
  File "c:\programdata\anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\programdata\anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
 

In [ ]:
epochs = 3
batch_size = 512 # Entire training set
model_path_unpruned = "Model_Saves/Unpruned.ckpt"
model_path_pruned = "Model_Saves/Pruned.ckpt"
NUM_CLASS = 10
tf.reset_default_graph()

# Import dataset
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
batches = int(len(mnist.train.images) / batch_size)

# Define Placeholders
# image = tf.placeholder(tf.float32, [None, 28,28])
# label = tf.placeholder(tf.float32, [None, 10])

image = tf.placeholder(name='images', dtype=tf.float32, shape=[None, 28, 28, 1])
label = tf.placeholder(name='fine_labels', dtype=tf.int32, shape=[None,10])

# # Define the model
# layer1 = layers.masked_fully_connected(image, 300)
# layer2 = layers.masked_fully_connected(layer1, 300)
# logits = layers.masked_fully_connected(layer2, 10)

# Define the model
# layer1 = layers.masked_conv2d(image, 300, kernel_size=2)
# layer2 = layers.masked_conv2d(layer1, 300, kernel_size=2)
# logits = layers.masked_fully_connected(layer2, 10)

_=image
_ = layers.masked_conv2d(_, 96, (3, 3), 1, 'SAME')
_ = tf.layers.batch_normalization(_, name='norm1-1')
_ = layers.masked_conv2d(_, 96, (3, 3), 1, 'SAME')
_ = tf.layers.batch_normalization(_, name='norm1-2')
_ = tf.layers.max_pooling2d(_, (3, 3), 2, 'SAME',name='pool1')
_ = layers.masked_conv2d(_, 192, (3, 3), 1, 'SAME')
_ = tf.layers.batch_normalization(_, name='norm2-1')
_ = layers.masked_conv2d(_, 192, (3, 3), 1, 'SAME')
_ = tf.layers.batch_normalization(_, name='norm2-2')
_ = tf.layers.max_pooling2d(_, (3, 3), 2, 'SAME', name='pool2')
_ = layers.masked_conv2d(_, 192, (3, 3), 1, 'VALID')
_ = tf.layers.batch_normalization(_, name='norm3')
_ = layers.masked_conv2d(_, 192, (1, 1), 1)
_ = tf.layers.batch_normalization(_, name='norm4')
_ = layers.masked_conv2d(_, 10, (1, 1), 1)
_ = tf.layers.batch_normalization(_, name='norm5')
_ = tf.layers.average_pooling2d(_, (5,5), 1, name='avg_pool')
y = _
logits = tf.reshape(y,[tf.shape(y)[0],10])


# Create global step variable (needed for pruning)
global_step = tf.train.get_or_create_global_step()
reset_global_step_op = tf.assign(global_step, 0)

# Loss function
#loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=label))
#loss = tf.losses.softmax_cross_entropy(tf.one_hot(label, NUM_CLASS), logits)
loss = tf.losses.softmax_cross_entropy(label, logits)

# Training op, the global step is critical here, make sure it matches the one used in pruning later
# running this operation increments the global_step
train_op = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(loss, global_step=global_step)

# Accuracy ops
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(label, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))



In [ ]:
# Get, Print, and Edit Pruning Hyperparameters
pruning_hparams = pruning.get_pruning_hparams()
print("Pruning Hyperparameters:", pruning_hparams)

# Change hyperparameters to meet our needs
pruning_hparams.begin_pruning_step = 0
pruning_hparams.end_pruning_step = 250
pruning_hparams.pruning_frequency = 1
pruning_hparams.sparsity_function_end_step = 250
pruning_hparams.target_sparsity = .5

# Create a pruning object using the pruning specification, sparsity seems to have priority over the hparam
p = pruning.Pruning(pruning_hparams, global_step=global_step) #sparsity=.5)
prune_op = p.conditional_mask_update_op()

# Create a saver for writing training checkpoints.
saver = tf.train.Saver()



In [ ]:
with tf.Session() as sess:

    # Uncomment the following if you don't have a trained model yet
    sess.run(tf.global_variables_initializer())

    # Train the model before pruning (optional)
    for epoch in range(epochs):
        for batch in range(batches):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            #print(batch_xs.shape)
            batch_xs = batch_xs.reshape(-1,28,28,1)
            sess.run(train_op, feed_dict={image: batch_xs, label: batch_ys})

        # Calculate Test Accuracy every 10 epochs
        if epoch % 1 == 0:
            acc_print = 0
            for batch in range(batches):
                batch_xt, batch_yt = mnist.test.next_batch(batch_size)
                #print(batch_xt.shape)
                batch_xt = batch_xt.reshape(-1,28,28,1)
                acc_print += sess.run(accuracy, feed_dict={image: batch_xt, label: batch_yt})
            print("Un-pruned model step %d test accuracy %g" % (epoch, acc_print/batches))
        print(epoch)
    
    # Saves the model before pruning
    saver.save(sess, model_path_unpruned)
    
    acc_print = sess.run(accuracy, feed_dict={image: mnist.test.images.reshape(-1,28,28,1), label: mnist.test.labels})
    print("Pre-Pruning accuracy:", acc_print)
    print("Sparsity of layers (should be 0)", sess.run(tf.contrib.model_pruning.get_weight_sparsity()))

    

    # Resets the session and restores the saved model
    sess.run(tf.initialize_all_variables())
    saver.restore(sess, model_path_unpruned)

    # Reset the global step counter and begin pruning
    sess.run(reset_global_step_op)
    for epoch in range(epochs):
        for batch in range(batches):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            batch_xs = batch_xs.reshape(-1,28,28,1)
            # Prune and retrain
            sess.run(prune_op)
            sess.run(train_op, feed_dict={image: batch_xs, label: batch_ys})

        # Calculate Test Accuracy every 10 epochs
        if epoch % 1 == 0:
            acc_print = 0
            for batch in range(batches):
                batch_xt, batch_yt = mnist.test.next_batch(batch_size)
                #print(batch_xt.shape)
                batch_xt = batch_xt.reshape(-1,28,28,1)
                acc_print += sess.run(accuracy, feed_dict={image: batch_xt, label: batch_yt})
            
            print("Pruned model step %d test accuracy %g" % (epoch, acc_print/batches))
            print("Weight sparsities:", sess.run(tf.contrib.model_pruning.get_weight_sparsity()))
        print(epoch) 
        
           # acc_print = sess.run(accuracy, feed_dict={image: mnist.test.images, label: mnist.test.labels})

    # Saves the model after pruning
    saver.save(sess, model_path_pruned)

    # Print final accuracy
    #acc_print = sess.run(accuracy, feed_dict={image: mnist.test.images, label: mnist.test.labels})
    print("Final accuracy:", acc_print)
    print("Final sparsity by layer (should be 0)", sess.run(tf.contrib.model_pruning.get_weight_sparsity()))

In [ ]:
mnist

In [ ]:
batches